In [3]:
from openpyxl import load_workbook
from openpyxl import Workbook
from openpyxl.styles import NamedStyle, Font, PatternFill, Alignment
from openpyxl.worksheet.formula import ArrayFormula

import os

# Die folgenden imports stammen aus der Datei ../00_common/common.py

import sys
sys.path.append("../00_common")
import common

def check_fingerprint(wb, matr):
    try: 
        ws = wb["1) ABC-Analyse"]
        if ws["U12"].value != None:
            if ws["U12"].value == int(matr):
                return "OK"
        else:
            return "NOK"
    except:
        print("FAIL:", matr, "check_fingerprint")
        return "NOK"

def check_arrayformula(wb, matr):
    try:
        response = "     "
        ws = wb.worksheets[0]
        if ws.array_formulae != {}:
            response = response[:1] + "1" + response[2:]
        
        ws = wb.worksheets[1]
        if ws.array_formulae != {}:
            response = response[:3] + "2" + response[4:]

        return response
    except:
        print("FAIL:", matr, "check Matrix_Formula")
        return "NOK"


def check_exercise_1_1(wb, matr):
    # Check formatting of cells A1:K1  (only check B1 and J1)
    try:
        ws = wb["1) ABC-Analyse"]
        points = 0
        # Check if background color is FH-Mint
        if ws["B1"].fill.start_color.index == ws["J1"].fill.start_color.index == "FF00B1AC":
            points += 1
    
        # Check, if the numberformat in cells B3 and C10 has one number after the decimal point
        if ws["B3"].number_format == r'_-* #,##0.00\ "€"_-;\-* #,##0.00\ "€"_-;_-* "-"??\ "€"_-;_-@_-': # Excel dt "Buchhaltung"
            points += 1
        if ws["B3"].number_format == r'#,##0.00\ "€"':                                                  # Excel dt "Währung"
            points += 1
        if ws["B3"].number_format == r'0.00 "€"':
            points += 1
        if ws["C10"].number_format == r'_-* #,##0.00\ "€"_-;\-* #,##0.00\ "€"_-;_-* "-"??\ "€"_-;_-@_-': # Excel dt "Buchhaltung"
            points += 1
        if ws["C10"].number_format == r'#,##0.00\ "€"':                                                  # Excel dt "Währung"
            points += 1
        if ws["C10"].number_format == r'0.00 "€"':
            points += 1  
    
        # Check if number-format in column A is "Kund-"0000
        for i in range(3, ws.max_row):
            cell = "A" + str(i)
            if ws[cell].number_format == r'"Kund-"0000':
                points += 1
                break
        #if ws["A10"].number_format == r'"Kund-"0000':
        #    points += 1 
            
        return points
    except:
        print("FAIL:", matr, "exercise 1_1")
        return "NOK"

def check_exercise_1_2(wb, matr):
    # Check, if turnover data is sorted by turnover
    try:
        # find real max_row
        ws = wb["1) ABC-Analyse"]
        max_row = 1
        for i in range(1, ws.max_row + 1):
            cell = "A" + str(i)
            if  ws[cell].value != None:
                max_row = i
        
        l_turnover = []
        for row in range(2, max_row + 1):
            cellname = "B"+str(row)
            l_turnover.append(ws[cellname].value)
        control_list = sorted(l_turnover, reverse = True)
        if l_turnover == control_list:
            return 2
        else: 
            return 0
    except:
        print("FAIL:", matr, "exercise 1_2")
        return "NOK"

def check_exercise_1_3(wb, wb_val, matr):
    # Check, if cumulated column contains the sum of all original values in the last cell and the max value in the first cell
    try:
        # find real max_row
        ws = wb["1) ABC-Analyse"]
        max_row = 1
        for i in range(1, ws.max_row + 1):
            cell = "A" + str(i)
            if  ws[cell].value != None:
                max_row = i
                
        ws_val = wb_val["1) ABC-Analyse"]
        l_turnover = []
        for row in range(2, max_row + 1):
            cellname = "B"+str(row)
            l_turnover.append(ws[cellname].value)
        cellname = "C" + str(max_row)
        if ws_val["C2"].value != None and ws_val[cellname].value != None:
            if round(ws_val["C2"].value, 2) == round(max(l_turnover), 2) and round(ws_val[cellname].value, 2) == round(sum(l_turnover), 2):
                return 2
            else:
                return 0
        else:
            return 0
    except:
        print("FAIL:", matr, "exercise 1_3")
        return "NOK"

def check_exercise_1_4(wb, matr):
    # Check, if formula for cumulative values is used with autofill, check, if there is a $
    try:
        ws = wb["1) ABC-Analyse"]
        if ws["C3"].value != None:
            formula = ws["C3"].value
            if ("C2" in formula or "$C2" in formula) and ("B3" in formula or "$B3" in formula):
                return 2
            elif ("B$2" in formula or "$B$2" in formula) and ("B3" in formula or "$B3" in formula) and ("SUM" in formula):
                return 2
            
            else:
                return 0
        return 0
    except:
        print("FAIL:", matr, "exercise 1_4")
        return "NOK"

def check_exercise_1_5(wb, matr):
    # Check if the class (A, B, C) is calculated using the VLOOKUP() function.
    try:
        ws = wb["1) ABC-Analyse"]

        # Check, if Array-Formulas are used
        if ws.array_formulae != {}:
            if "E2" in ws.array_formulae:
                formula = ws["E2"].value.text
            else:
                formula = ws["E2"].value
        elif ws["E2"].value != None:
                formula = ws["E2"].value
        else:
            return 0
        
        formula = formula.replace(" ", "")
        if formula[:8] == "=VLOOKUP" and formula.count("$") >= 1:
            return 2
        elif formula[:3] == "=IF" and formula.count("$") >= 1:
            return 1
        else:
            return 0
        return 10    
    except:
        print("FAIL:", matr, "exercise 1_5")
        return "NOK"

def check_exercise_1_6(wb_val, matr):
    # Check if the values for the calculation of the ABC-Analysis are correct
    try:
        # find real max_row
        ws = wb_val["1) ABC-Analyse"]
        max_row = 1
        for i in range(1, ws.max_row + 1):
            cell = "A" + str(i)
            if  ws[cell].value != None:
                max_row = i
        
        l_turnover = []
        for row in range(2, max_row + 1):
            cellname = "B"+str(row)
            l_turnover.append(ws[cellname].value)
        l_turnover = sorted(l_turnover, reverse = True)
        sum_turnover = sum(l_turnover)
        accumulated = 0
        l_abc = []

        
        if ws["G3"].value != None:
            AB_border = ws["G3"].value
        else:
            return 0
        if ws["G4"].value != None:
            BC_border = ws["G4"].value
        else:
            return 0

        for turnover in l_turnover:
            accumulated += turnover
            accumulated_perc = accumulated / sum_turnover
            if accumulated_perc <= AB_border:
                abc_class = "A"
            elif accumulated_perc <= BC_border:
                abc_class = "B"
            else:
                abc_class = "C"
            l_abc.append((turnover, accumulated, accumulated_perc, abc_class))
        abc_count = {"A" : 0, "B" : 0, "C": 0}
        abc_turnover = {"A" : 0, "B" : 0, "C": 0}
        for cust in l_abc:
            if cust[3] == "A":
                abc_count["A"] += 1
                abc_turnover["A"] += cust[0]
            elif cust[3] == "B":
                abc_count["B"] += 1
                abc_turnover["B"] += cust[0]
            elif cust[3] == "C":
                abc_count["C"] += 1
                abc_turnover["C"] += cust[0]
    
        abc_turnover["A"] = round(abc_turnover["A"], 2)
        abc_turnover["B"] = round(abc_turnover["B"], 2)
        abc_turnover["C"] = round(abc_turnover["C"], 2)
    
        points = 0

        # Check for Array-Formulas not required as only ws_values are opened
        if ws["J2"].value != None and ws["J3"].value != None and ws["J4"].value != None:
            if ws["J2"].value == abc_count["A"] and ws["J3"].value == abc_count["B"] and ws["J4"].value == abc_count["C"]:
                points += 2
        if ws["K2"].value != None and ws["K3"].value != None and ws["K4"].value != None:
            if round(ws["K2"].value, 2) == abc_turnover["A"] and round(ws["K3"].value, 2) == abc_turnover["B"] and round(ws["K4"].value, 2) == abc_turnover["C"]:
                points += 2
        
        return points
    except:
        print("FAIL:", matr, "exercise 1_6")
        return "NOK"

def check_exercise_2_1(wb, matr):
    # Check formatting of cells A1:K1  (only check B1 and J1)
    try:
        ws = wb["2) Klausur"]
        points = 0
        # Check if background color is FH-Mint
        if ws["D2"].fill.start_color.index == ws["J4"].fill.start_color.index == ws["P2"].fill.start_color.index == "FF00B1AC":
            points += 1
    
        # Check, if font-size in row 1 is 16
        if ws["A1"].font.size == ws["N1"].font.size == 16:
            points += 1
    
        return points
    except:
        print("FAIL:", matr, "exercise 2_1")
        return "NOK"

def check_exercise_2_2(wb, matr):
    # Check, if stud-ids (matr) are sorted
    try:
        # find real max_row
        ws = wb["2) Klausur"]
        max_row = 1
        for i in range(1, ws.max_row + 1):
            cell = "A" + str(i)
            if  ws[cell].value != None:
                max_row = i
        
        l_matr = []
        for row in range(3, max_row + 1):
            cellname = "A"+str(row)
            if ws[cellname].value != None:
                l_matr.append(ws[cellname].value)
        control_list = sorted(l_matr)
        if l_matr == control_list and len(l_matr) == max_row - 2:
            return 2
        
        l_matr = []
        for row in range(4, max_row + 1):
            cellname = "A"+str(row)
            if ws[cellname].value != None:
                l_matr.append(ws[cellname].value)
        control_list = sorted(l_matr)
        if l_matr == control_list and len(l_matr) == max_row - 3:
            return 2
        else: 
            return 0
    except:
        print("FAIL:", matr, "exercise 2_2")
        return "NOK"

def check_exercise_2_3(wb, matr):
    # Check if the VLOOKUP Function is used correctly. Check Formula in Cell D6
    try:
        ws = wb["2) Klausur"]
        points = 0
        # Check, if Array-Formulas are used
        if "D3" in ws.array_formulae: 
            formula = ws["D3"].value.text
            if "VLOOKUP" in formula:
                points += 1
            formula = formula.split(",")
            if "A" in formula[0][9:] and "3" in formula[0][9:]:
                points += 1
            if ("N" in formula[1] and "T" in formula[1]) or ("N" in formula[1] and "AH" in formula[1]):
                points += 1
            if formula[2] == "7":
                points += 1
            if formula[3] == "FALSE)":
                points += 1
            return points

        else: # No array formula used for column D
            if ws["D6"].value != None:
                formula = ws["D6"].value 
                if "VLOOKUP" in formula:
                    points += 1
                formula = formula.split(",")
                if formula[0][9:] == "A6":
                    points += 1
                elif formula[0][9:] == "$A6":
                    points += 1
                elif formula[0][9:] == "$A$6":
                    points += 1
                if formula[1] == "N:T" or formula[1] == "N:AH" or formula[1].count("$") >= 2:
                    points += 1
                if formula[2] == "7":
                    points += 1
                if formula[3] == "FALSE)":
                    points += 1
            return points
    except:
        print("FAIL:", matr, "exercise 2_3")
        return "NOK"
    
def check_exercise_2_4(wb, matr):
    # Check if the VLOOKUP Function is used correctly. Check Formula in Cell H11
    try:
        ws = wb["2) Klausur"]
        points = 0
        # Check, if Array-Formulas are used
        if "H3" in ws.array_formulae: 
            formula = ws["H3"].value.text
            if "VLOOKUP" in formula:
                points += 1
            formula = formula.split(",")
            if "G" in formula[0][9:] and "3" in formula[0][9:]:
                points += 1
            if ("J" in formula[1] and "L" in formula[1]) or ("J" in formula[1] and "K" in formula[1]):
                points += 1
            if len(formula) == 3:
                points += 1
            elif formula[3] == "TRUE)":
                points += 1
            return points
            
        else:       
            if ws["H11"].value != None:
                formula = ws["H11"].value 
                if "VLOOKUP" in formula:
                    points += 1
                formula = formula.split(",")
                if formula[0][9:] == "G11" or formula[0][9:] == "$G11":
                    points += 1
                if formula[1] == "J:L" or  formula[1] == "J:K" or formula[1].count("$") >= 2:
                    points += 1
                if len(formula) == 3:
                    points += 1
                elif formula[3] == "TRUE)":
                    points += 1
            return points
    except:
        print("FAIL:", matr, "exercise 2_4")
        return "NOK"

def check_exercise_2_5(wb, matr):
    # Check if the Hilfstabelle, column J is designed correctly
    try:
        ws = wb["2) Klausur"]
        points = 0
        if ws["J5"].value == 0:
            points += 1
        if ws["J6"].value in ["=K2", "=$K$2", "=$K2", "=K$2"]:
            points += 10
        if ws["J7"].value in ["=J6+$K$3", "=J6+K$3", "=$J6+$K$3", "=$J6+K$3"]:
            points += 100
        return points
    except:
        print("FAIL:", matr, "exercise 2_5")
        return "NOK"

def check_exercise_2_6(wb, matr):
    # Check if the Hilfstabelle uses COUNTIF() in column L
    try:
        ws = wb["2) Klausur"]
        points = 0

        if "L5" in ws.array_formulae: 
            formula = ws["L5"].value.text
            if "COUNTIF" in formula:
                points += 1
            if "K5" in formula or "K$5" in formula:
                points += 1
            return points
        else: 
            if ws["L8"].value != None:
                if "COUNTIF" in ws["L8"].value:
                    points += 1
                if "K8" in ws["L8"].value:
                    points += 1
            return points
    except:
        print("FAIL:", matr, "exercise 2_6")
        return "NOK"


         
def check_workbook(matr, wb, wb_val):
    points = [matr]
    points.append(check_fingerprint(wb, matr))
    points.append(check_arrayformula(wb, matr))
    points.append(check_exercise_1_1(wb, matr))
    points.append(check_exercise_1_2(wb, matr))
    points.append(check_exercise_1_3(wb, wb_val, matr))
    points.append(check_exercise_1_4(wb, matr))
    points.append(check_exercise_1_5(wb, matr))
    points.append(check_exercise_1_6(wb_val, matr))
    points.append(check_exercise_2_1(wb, matr))
    points.append(check_exercise_2_2(wb, matr))
    points.append(check_exercise_2_3(wb, matr))
    points.append(check_exercise_2_4(wb, matr))
    points.append(check_exercise_2_5(wb, matr))
    points.append(check_exercise_2_6(wb, matr))

    print(points)
    
    return points



l_points = common.check_ExcelFiles_in_Abgaben(check_workbook)
header = ["MatrNr", "Fingerprint", "MatrixF", "1 Format", "1 Sortiert", "1 Kumluiert", "1 Autofill", "1 SVERWEIS", "1 ABC-Tabelle", "2 Format", "2 Sortiert", "2 SVW1", "2 SVW2", "2 Hilfst", "2 Countif"] 
common.result_into_file(l_points, header)


[3596865, 'OK', '     ', 4, 2, 2, 2, 2, 4, 2, 2, 5, 4, 11, 2]


In [ ]:
os.chdir(".")
print(os.getcwd())